In [2]:
import pandas as pd

In [5]:
df = pd.read_csv('http://bit.ly/kaggletrain')
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [8]:
df.shape

(891, 12)

In [11]:
df.columns

Index(['PassengerId', 'Survived', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp',
       'Parch', 'Ticket', 'Fare', 'Cabin', 'Embarked'],
      dtype='object')

In [14]:
df.isna().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

In [22]:
df = df.loc[df.Embarked.notna(), ['Survived', 'Pclass', 'Sex', 'Embarked']]

In [29]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
X = df.loc[:, ['Pclass']]
y = df.Survived
logreg = LogisticRegression(solver='lbfgs')
cross_val_score(logreg, X, y, cv=5, scoring='accuracy').mean()

0.6783406335301212

In [30]:
y.value_counts(normalize=True)

0    0.617548
1    0.382452
Name: Survived, dtype: float64

In [32]:
from sklearn.preprocessing import OneHotEncoder
ohe = OneHotEncoder()

In [33]:
ohe.fit_transform(df[['Sex']])

<889x2 sparse matrix of type '<class 'numpy.float64'>'
	with 889 stored elements in Compressed Sparse Row format>

In [34]:
ohe.categories_

[array(['female', 'male'], dtype=object)]

In [37]:
X = df.drop('Survived', axis='columns')

In [38]:
X.head()

,Pclass,Sex,Embarked
0,3,male,S
1,1,female,C
2,3,female,S
3,1,female,S
4,3,male,S


In [41]:
from sklearn.compose import make_column_transformer
column_trans = make_column_transformer(
    (OneHotEncoder(),  ['Sex', 'Embarked']),
    remainder='passthrough'
)
column_trans.fit_transform(X)

array([[0., 1., 0., 0., 1., 3.],
       [1., 0., 1., 0., 0., 1.],
       [1., 0., 0., 0., 1., 3.],
       ...,
       [1., 0., 0., 0., 1., 3.],
       [0., 1., 1., 0., 0., 1.],
       [0., 1., 0., 1., 0., 3.]])

In [53]:
from sklearn.pipeline import make_pipeline
from sklearn.tree import DecisionTreeClassifier
clf = DecisionTreeClassifier(random_state=42)
pipe = make_pipeline(column_trans, clf)
cross_val_score(pipe, X, y, cv=5, scoring='accuracy').mean()

0.8110264711483527

In [54]:
pipe.fit(X, y)
importances = clf.feature_importances_

In [55]:
importances

array([0.71061814, 0.        , 0.00677993, 0.00158283, 0.04338738,
       0.23763173])

In [ ]:
colors = ['#DF2020', '#81DF20', '#2095DF']